#### Prepared for Gabor's Data Analysis

### Data Analysis for Business, Economics, and Policy
by Gabor Bekes and  Gabor Kezdi
 
Cambridge University Press 2021

**[gabors-data-analysis.com ](https://gabors-data-analysis.com/)**

 License: Free to share, modify and use for educational purposes. 
 Not to be used for commercial purposes.

Stock market regression
chapter 12
sandp-stocks

Loading and cleaning data

In [6]:
# import packages
import pandas as pd
import os

In [7]:
# set your directory here
data_dir = "/Users/vigadam/Dropbox/work/data_book/da_data_repo"

data_in = data_dir + "/stocks-sp500/raw/"
data_out = data_dir + "/stocks-sp500/clean/"

In [8]:
# load and extract MSFT
df = pd.read_csv(
    data_in + "ready_sp500_45_cos.csv", quotechar='"', delimiter=",", encoding="utf-8"
)

df = df.loc[df.ticker == "MSFT"].reset_index()
df["date"] = pd.to_datetime(df["ref.date"])
df["p_MSFT"] = df["price.close"]
#df = df.round({"p_MSFT": 2})
df_MSFT = df[["date", "p_MSFT"]]

In [9]:
# load and extract SP500
df = pd.read_csv(
    data_in + "ready_sp500_index.csv", quotechar='"', delimiter=",", encoding="utf-8"
)
df["date"] = pd.to_datetime(df["ref.date"])
df["p_SP500"] = df["price.close"]
#df = df.round({"p_SP500": 2})
df_SP500 = df[["date", "p_SP500"]]

In [10]:
# merge data and filter by date
df = pd.merge(df_SP500, df_MSFT, how="inner", on="date")
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["ym"] = pd.to_datetime(df[["year", "month"]].assign(day=1)).dt.to_period("M")
order = ["ym", "year", "month", "date"]
df = df.reindex(
    columns=order + [column for column in df.columns if column not in order]
)
df.drop(df.loc[df.date < "1997-12-31"].index, inplace=True)
df.drop(df.loc[df.date > "2018-12-31"].index, inplace=True)

df = df.reset_index(drop=True)

df.to_csv(data_out + "stock-prices-daily.csv", index=False)